<a href="https://colab.research.google.com/github/Torsha-Sett/Toxic-comment-challenge/blob/main/Word2Vec_Glove_CuDNNLSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Toxic comment challenge/

/content/gdrive/MyDrive/Toxic comment challenge


In [ ]:
import sys, os, re, csv, codecs, numpy as np, pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation
from keras.layers import CuDNNLSTM, CuDNNGRU
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model
#from keras import initializers, regularizers, constraints, optimizers, layers
import matplotlib.pyplot as plt
%matplotlib inline
import gensim.models.keyedvectors as word2vec
import pandas as pd
import gc
from tqdm import tqdm

In [ ]:
train = pd.read_csv("train.csv.zip")
test = pd.read_csv("test.csv.zip")
test_y = pd.read_csv("test_labels.csv.zip")

In [ ]:
embed_size=0
list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y = train[list_classes].values
list_sentences_train = train["comment_text"]
list_sentences_test = test["comment_text"]

In [ ]:
max_features = 20000
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(list(list_sentences_train))
list_tokenized_train = tokenizer.texts_to_sequences(list_sentences_train)
list_tokenized_test = tokenizer.texts_to_sequences(list_sentences_test)

In [ ]:
def loadEmbeddingMatrix(typeToLoad):
        # load different embedding file from Kaggle depending on which embedding 
        # matrix we are going to experiment with
        if(typeToLoad=="glove"):
            #EMBEDDING_FILE='../input/glove-twitter/glove.twitter.27B.25d.txt'
            embeddings_index_ = pd.read_pickle('../input/pickled-glove840b300d-for-10sec-loading/glove.840B.300d.pkl')
            embeddings_index = {key:value for key, value in embeddings_index_.items() if len(value)==300}
            print('Loaded %s word vectors.' % len(embeddings_index))
            embed_size = 300
        elif(typeToLoad=="word2vec"):
            word2vecDict = word2vec.KeyedVectors.load_word2vec_format("../input/googlenewsvectorsnegative300/GoogleNews-vectors-negative300.bin", binary=True)
            embed_size = 300
        elif(typeToLoad=="fasttext"):
            EMBEDDING_FILE='../input/fasttext/wiki.simple.vec'
            embed_size = 300

        if(typeToLoad=="fasttext" ):#typeToLoad=="glove" or 
            embeddings_index = dict()
            # Transfer the embedding weights into a dictionary by iterating through every line of the file.
            f = open(EMBEDDING_FILE)
            for line in tqdm(f):
                try:
                    # split up line into an indexed array
                    values = line.split()
                    # first index is word
                    word = values[0]
                    # store the rest of the values in the array as a new array
                    coefs = np.asarray(values[1:], dtype='float32')
                    # ignore values without headers as word
                    if len(coefs)==300:
                        embeddings_index[word] = coefs #300 dimension
                    else:
                        continue
                except:
                    # split up line into an indexed array
                    values = line.split()
                    # first index is word or sometimes the second as well
                    word = values[1]
                    # store the rest of the values in the array as a new array
                    coefs = np.asarray(values[2:], dtype='float32')
                    print(values[:2])
                    print(len(coefs))
                    embeddings_index[word] = coefs #300 dimension
            f.close()
            print('Loaded %s word vectors.' % len(embeddings_index))
            #return embeddings_index
        elif typeToLoad=="word2vec":
            embeddings_index = dict()
            for word in word2vecDict.wv.vocab:
                embeddings_index[word] = word2vecDict.word_vec(word)
            print('Loaded %s word vectors.' % len(embeddings_index))
            
        gc.collect()
        # We get the mean and standard deviation of the embedding weights so that we could maintain the 
        # same statistics for the rest of our own random generated weights. 
        all_embs = np.stack(embeddings_index.values())
        #np.stack(list(embeddings_index.values()))
        emb_mean, emb_std = all_embs.mean(), all_embs.std()
        
        nb_words = len(tokenizer.word_index)
        # We are going to set the embedding size to the pretrained dimension as we are replicating it.
        # the size will be Number of Words in Vocab X Embedding Size
        embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
        gc.collect()

        # With the newly created embedding matrix, we'll fill it up with the words that we have in both 
        # our own dictionary and loaded pretrained embedding. 
        embeddedCount = 0
        for word, i in tokenizer.word_index.items():
            i-=1
            # then we see if this word is in glove's dictionary, if yes, get the corresponding weights
            embedding_vector = embeddings_index.get(word)
            # and store inside the embedding matrix that we will train later on.
            if embedding_vector is not None: 
                embedding_matrix[i] = embedding_vector
                embeddedCount+=1
        print('total embedded:',embeddedCount,'common words')
        
        del(embeddings_index)
        gc.collect()
        
        # finally, return the embedding matrix
        return embedding_matrix

ValueError: ignored

In [ ]:
embedding_matrix = loadEmbeddingMatrix('word2vec')

In [ ]:
inp = Input(shape=(maxlen, )) #maxlen=200 as defined earlier

x = Embedding(len(tokenizer.word_index), embedding_matrix.shape[1], weights=[embedding_matrix], trainable=True)(inp)

#x = Bidirectional(LSTM(60, return_sequences=True, name='lstm_layer', dropout=0.1, recurrent_dropout=0.1))(x)
x = Bidirectional(CuDNNLSTM(60, return_sequences=True, name='lstm_layer'))(x)
x = Dropout(0.1)(x)


In [ ]:
x = GlobalMaxPool1D()(x)
x = Dropout(0.1)(x)
x = Dense(50, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(6, activation="sigmoid")(x)


In [ ]:
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

model.summary()

In [ ]:
# some predictions
print(list_sentences_test[10])
print(dict(zip(list_classes, model.predict(X_te[10:11])[0])))
print()
print()
print(list_sentences_test[1001])
print(dict(zip(list_classes, model.predict(X_te[1001:1002])[0])))
print()
print()

# training data
i=2018
print(list_sentences_train[i])
print(dict(zip(list_classes, model.predict(X_t[i:i+1])[0])))
print('Actual Values:', train[list_classes].iloc[i].to_dict())
print()
print()
i=159554
print(list_sentences_train[i])
print(dict(zip(list_classes, model.predict(X_t[i:i+1])[0])))
print('Actual Values:', train[list_classes].iloc[i].to_dict())

In [ ]:
epochRange = np.arange(1,5,1)
plt.plot(epochRange,all_losses['baseline_loss'])
plt.plot(epochRange,all_losses['baseline_val_loss'])
plt.title('Training Vs Validation loss for baseline model')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['Training', 'Validation'], loc='upper left')
plt.show()